In [1]:
import numpy as np 
import cv2 
import pandas as pd
import glob
import time 
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

def extract_sift_des(image):
    sift = cv2.ORB_create()
    kp, des = sift.detectAndCompute(image, None)
    return des

def read_bounding_box(path):
    boundary_file = open(path, 'r')
    boundary = boundary_file.readline().strip().split(' ')
    boundary = [int(b) for b in boundary]
    boundary_file.close()
    return boundary

In [2]:
download_path='C:/Users/IV/Desktop/CityU' # change to your own download path
path_query=download_path+'/query_4186'
path_query_txt=download_path+'/query_txt_4186'

# path_query_txt is the directory to the bounding box information of the instance(s) for the query images
path_gallery=download_path+'/gallery_4186'

name_query=glob.glob(path_query+'/*.jpg')
num_query=len(name_query)
name_gallery=glob.glob(path_gallery+'/*.jpg')
num_gallery=len(name_gallery)
record_all=np.zeros((num_query,len(name_gallery)))
query_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_query+'/*.jpg')]
query_imgs_no = [x[:-4] for x in query_imgs_no]

gallery_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_gallery+'/*.jpg')]
gallery_imgs_no = [x[:-4] for x in gallery_imgs_no]

gallery_des = []

for i, gallery_img_no in tqdm(enumerate(gallery_imgs_no)):
    per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
    per_gallery=cv2.imread(per_gallery_name)
    # Image pre-processing
    per_gallery = cv2.GaussianBlur(per_gallery,(3,3),0)
    per_gallery = cv2.cvtColor(per_gallery,cv2.COLOR_BGR2YUV)
    per_gallery[:,:,0] = cv2.equalizeHist(per_gallery[:,:,0])
    per_gallery = cv2.cvtColor(per_gallery,cv2.COLOR_YUV2BGR)
    # feature extraction for per gallery
    per_gallery_des = extract_sift_des(per_gallery)
    gallery_des.append(per_gallery_des)

5000it [03:03, 27.19it/s]


In [3]:
for i, query_img_no in tqdm(enumerate(query_imgs_no[0:20])):
    time_s = time.time()
    dist_record=[]
    gallery_imgs_no_desc=[]
    per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
    per_query=cv2.imread(per_query_name)
    
    # read boundary from text file
    queryfilename = path_query_txt+'/'+str(query_img_no)+'.txt'
    
    # crop the image
    boundary = read_bounding_box(queryfilename)
    x ,y, w, h = boundary
    query_boundary = per_query[y:y+h, x:x+w]

    # Image pre-processing
    query_boundary = cv2.GaussianBlur(query_boundary,(3,3),0)
    query_boundary=cv2.cvtColor(query_boundary,cv2.COLOR_BGR2YUV)
    query_boundary[:,:,0] = cv2.equalizeHist(query_boundary[:,:,0])
    query_boundary = cv2.cvtColor(query_boundary,cv2.COLOR_YUV2BGR)

    # feature extraction for per query
    per_query_des = extract_sift_des(query_boundary)

    # the iteration loop for gallery
    for j, gallery_img_no in tqdm(enumerate(gallery_imgs_no), desc=f"Processing query part {i}"):
        # Compute similarity score
        sim_score = cosine_similarity(per_query_des, gallery_des[j])
        dist_record.append(sim_score)

    # find the indexes with descending similarity order
    ascend_index=sorted(range(len(dist_record)), key=lambda k: np.max(dist_record[k]),reverse=True)

    # update the results for one query
    for k in range(len(ascend_index)):
        gallery_imgs_no_desc.append(np.array(gallery_imgs_no)[ascend_index[k]])

    record_all[i,:]= gallery_imgs_no_desc
    time_e = time.time()
    print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))
    query_idx = i
    print(f'For query image No. {query_imgs_no[query_idx]}, the top 10 ranked similar image No. are {gallery_imgs_no_desc[0]} {gallery_imgs_no_desc[1]} {gallery_imgs_no_desc[2]} {gallery_imgs_no_desc[3]} {gallery_imgs_no_desc[4]} {gallery_imgs_no_desc[5]} {gallery_imgs_no_desc[6]} {gallery_imgs_no_desc[7]} {gallery_imgs_no_desc[8]} {gallery_imgs_no_desc[9] }')

Processing query part 0: 5000it [00:11, 450.85it/s]
1it [00:19, 19.09s/it]

retrieval time for query 1258 is 19.088042974472046s
For query image No. 1258, the top 10 ranked similar image No. are 2403 3831 2726 3960 91 3612 2408 734 3679 4937


Processing query part 1: 5000it [00:10, 486.14it/s]
2it [00:37, 18.56s/it]

retrieval time for query 1656 is 18.18750834465027s
For query image No. 1656, the top 10 ranked similar image No. are 2003 339 3748 4042 4209 3034 1938 3945 772 1227


Processing query part 2: 5000it [00:09, 509.96it/s]
3it [00:55, 18.20s/it]

...

retrieval time for query 4716 is 18.149714469909668s
For query image No. 4716, the top 10 ranked similar image No. are 4286 4184 1884 2807 4332 1459 654 691 3345 465


Processing query part 18: 5000it [00:09, 505.57it/s]
19it [05:29, 17.66s/it]

retrieval time for query 4929 is 17.95368456840515s
For query image No. 4929, the top 10 ranked similar image No. are 4080 3661 2238 3945 123 2138 4964 655 3749 4358


Processing query part 19: 5000it [00:06, 815.22it/s]
20it [05:43, 17.16s/it]

retrieval time for query 776 is 13.771233320236206s
For query image No. 776, the top 10 ranked similar image No. are 110 1583 3219 504 2292 4456 1032 2427 647 3460


In [4]:
# write the output file following the example
f=open(r'./rank_list_SIFT.txt','w')
for i in range(num_query):
    f.write('Q'+str(i+1)+': ')
    for j in range(len(name_gallery)):
        f.write(str(np.int32(record_all[i,j]))+' ')
    f.write('\n')
f.close()